In [74]:
import pandas as pd
from The_PAM_Clustering.PAM import PAM

Загрузим датасет (в нем несколько листов, их надо объединить)

In [75]:
df = pd.concat(pd.read_excel('./data/tutors_full.xlsx', sheet_name=None), ignore_index=True)

In [76]:
df.describe()

,Column1,id
count,288724.00000,2.887230e+05
mean,58844.96080,2.840819e+06
std,48382.63658,1.373217e+06
min,0.00000,5.100000e+01
25%,19705.00000,1.229736e+06
50%,43765.50000,3.884913e+06
75%,95438.25000,3.914147e+06
max,167619.00000,3.915058e+06


In [77]:
df.head()

,Column1,author,date,discipline,id,mark,reply,text,tutorlink,tutorname
0,0.0,Юлия,2019-07-03,биология,3913905.0,лучше не бывает!,NaN,Очень грамотный и приятный преподаватель ) оче...,https://repetitors.info/repetitor/?p=SuhorukovaNA,Сухорукова Наталия Альбертовна
1,1.0,Ксения,2019-07-03,не указана,3913841.0,лучше не бывает!,NaN,"Прислушивается к потребностям, очень хорошо об...",https://repetitors.info/repetitor/?p=RebrovRU,Ребров Родион Юрьевич
2,2.0,Виктория,2019-07-03,физика,3913830.0,лучше не бывает!,NaN,Сын писал пробный ЕГЭ на 70 баллов. Стал заним...,https://repetitors.info/repetitor/?p=SadovnikovSV,Садовников Сергей Викторович
3,3.0,Жанна,2019-07-03,английский язык,3913781.0,лучше не бывает!,NaN,"Лидия замечательный преподаватель, пунктуальны...",https://repetitors.info/repetitor/?p=ErshovaLV5,Ершова Лидия Владимировна
4,4.0,Елена,2019-07-03,русский язык,3913776.0,лучше не бывает!,NaN,"Плюсы: Репетитор замечательный, легко нашла ко...",https://repetitors.info/repetitor/?p=NoskovaEI,Носкова Евгения Игоревна


Видно, что в нем 288724 строки. Преобразуем данные, оставим только поля которые потом будем использовать для подсчета расстояния.

In [78]:
df_dropped = df.drop(columns=['Column1', 'author', 'date', 'id', 'reply', 'text', 'tutorlink'])
df_dropped.head()

,discipline,mark,tutorname
0,биология,лучше не бывает!,Сухорукова Наталия Альбертовна
1,не указана,лучше не бывает!,Ребров Родион Юрьевич
2,физика,лучше не бывает!,Садовников Сергей Викторович
3,английский язык,лучше не бывает!,Ершова Лидия Владимировна
4,русский язык,лучше не бывает!,Носкова Евгения Игоревна


In [79]:
df_dropped.describe()

,discipline,mark,tutorname
count,288724,288723,288723
unique,77,10,40408
top,математика,ужасно,Земченко Флорина Викторовна
freq,70333,167620,8280


Поскольку мы кластеризуем репетиторов нам необходимо оставить только уникальных людей. Также конвертируем оценки в численное представление.

In [80]:
mark2num = {
    'лучше не бывает!': 10,
    'отлично': 9,
    'почти отлично': 8,
    'вполне хорошо': 7,
    'хорошо': 6,
    'почти хорошо': 5,
    'более-менее': 4,
    'так себе...': 3,
    'плохо': 2,
    'ужасно': 1
}
df_dropped.mark = df_dropped.mark.map(mark2num)
df_dropped.head()

,discipline,mark,tutorname
0,биология,10.0,Сухорукова Наталия Альбертовна
1,не указана,10.0,Ребров Родион Юрьевич
2,физика,10.0,Садовников Сергей Викторович
3,английский язык,10.0,Ершова Лидия Владимировна
4,русский язык,10.0,Носкова Евгения Игоревна


Сформируем таблицу с уникальными преподавателями. Будем использовать их среднюю оценку и наиболее частый предмет. Это предположение основано на том, что преподаватель, скорее всего будет преподавать один предмет (или хотя бы похожие), так что с этого мы ничего не потеряем

In [81]:
df_mean = df_dropped.groupby(['tutorname']).mean()
df_mean

,mark
tutorname,
3ыбина Анна Сергеевна,10.0
Aaron Hanisch,9.0
Aaron James Dodd,9.0
Aaron Michael Burhoe,8.0
Abala David Ogero,9.0
...,...
Ященко Анастасия Олеговна,9.0
Ященко Николай Владиславович,10.0
Ященко Юлия Владимировна,9.0


In [82]:
df_common_disc = df_dropped.groupby(['tutorname'])['discipline'].agg(pd.Series.mode).to_frame()
df_common_disc

,discipline
tutorname,
3ыбина Анна Сергеевна,рисование
Aaron Hanisch,английский язык
Aaron James Dodd,английский язык
Aaron Michael Burhoe,английский язык
Abala David Ogero,английский язык
...,...
Ященко Анастасия Олеговна,английский язык
Ященко Николай Владиславович,химия
Ященко Юлия Владимировна,история


In [83]:
df_use = df_mean.join(df_common_disc)
df_use

,mark,discipline
tutorname,,
3ыбина Анна Сергеевна,10.0,рисование
Aaron Hanisch,9.0,английский язык
Aaron James Dodd,9.0,английский язык
Aaron Michael Burhoe,8.0,английский язык
Abala David Ogero,9.0,английский язык
...,...,...
Ященко Анастасия Олеговна,9.0,английский язык
Ященко Николай Владиславович,10.0,химия
Ященко Юлия Владимировна,9.0,история


In [84]:
df_use = df_use.reset_index()
df_use

,tutorname,mark,discipline
0,3ыбина Анна Сергеевна,10.0,рисование
1,Aaron Hanisch,9.0,английский язык
2,Aaron James Dodd,9.0,английский язык
3,Aaron Michael Burhoe,8.0,английский язык
4,Abala David Ogero,9.0,английский язык
...,...,...,...
40403,Ященко Анастасия Олеговна,9.0,английский язык
40404,Ященко Николай Владиславович,10.0,химия
40405,Ященко Юлия Владимировна,9.0,история
40406,Ященкова Инна Александровна,10.0,музыка


Итак это мы и будем использовать. Описательные характеристики:

In [85]:
df_use.describe()

,mark
count,40408.000000
mean,8.889423
std,1.573467
min,1.000000
25%,9.000000
50%,9.200000
75%,10.000000
max,10.000000


Настало время кластеризовать, используя PAM. Также введем функцию для подсчета расстояния.

In [86]:
def my_tutor_distance(X, Y):
  result = 0
  if X['discipline'] != Y['discipline']:
    result += 3
  result += abs(X['mark'] - Y['mark'])
  return result

В целях экономии времени и памяти (ее, скорее всего, не хватит для всех) просемплируем данные.

In [98]:
df_sample = df_use.sample(frac=(1.0/80))
df_sample = df_sample.reset_index()
df_sample

,index,tutorname,mark,discipline
0,29897,Рязанский Валерий Павлович,9.000000,математика
1,25149,Новичкова Светлана Геннадьевна,9.444444,биология
2,5386,Будник Полина Петровна,5.666667,сольфеджио
3,36683,Хващинский Дмитрий Альбертович,1.000000,биология
4,32443,Соловьев Иван Николаевич,9.000000,математика
...,...,...,...,...
500,21572,Малый Сергей Николаевич,6.800000,математика
501,27273,Плотников Александр Сергеевич,9.000000,электротехника
502,17015,Конончик Ирина Олеговна,9.000000,английский язык
503,15193,Карташева Белла Анатольевна,10.000000,фортепиано


In [102]:
df_str = df_sample['discipline'].apply(str)
df_sample['discipline'] = df_str
df_sample = df_sample.drop(columns=['index'])

In [103]:
df_sample

,tutorname,mark,discipline
0,Рязанский Валерий Павлович,9.000000,математика
1,Новичкова Светлана Геннадьевна,9.444444,биология
2,Будник Полина Петровна,5.666667,сольфеджио
3,Хващинский Дмитрий Альбертович,1.000000,биология
4,Соловьев Иван Николаевич,9.000000,математика
...,...,...,...
500,Малый Сергей Николаевич,6.800000,математика
501,Плотников Александр Сергеевич,9.000000,электротехника
502,Конончик Ирина Олеговна,9.000000,английский язык
503,Карташева Белла Анатольевна,10.000000,фортепиано


In [104]:
df_sample.describe()

,mark
count,505.000000
mean,8.991547
std,1.413870
min,1.000000
25%,9.000000
50%,9.307692
75%,10.000000
max,10.000000


In [105]:
med, clusters, TD = PAM(df_sample, k=12, dist=my_tutor_distance, maxIter=2000)

In [106]:
# количество в разных кластерах
df_sample['cluster'] = clusters
df_sample['cluster'].value_counts()

30     113
0       97
13      76
40      39
232     36
474     27
125     25
316     24
184     22
33      19
165     14
238     13
Name: cluster, dtype: int64

In [107]:
df_sample

,tutorname,mark,discipline,cluster
0,Рязанский Валерий Павлович,9.000000,математика,0
1,Новичкова Светлана Геннадьевна,9.444444,биология,316
2,Будник Полина Петровна,5.666667,сольфеджио,125
3,Хващинский Дмитрий Альбертович,1.000000,биология,238
4,Соловьев Иван Николаевич,9.000000,математика,0
...,...,...,...,...
500,Малый Сергей Николаевич,6.800000,математика,125
501,Плотников Александр Сергеевич,9.000000,электротехника,0
502,Конончик Ирина Олеговна,9.000000,английский язык,30
503,Карташева Белла Анатольевна,10.000000,фортепиано,13


Это и есть результат кластеризации (напомним, что номер кластера - это индекс его представителя). При желании анализ можно продолжить в желаемом направлении. Вот, например, центры кластеров.

In [108]:
df_sample.iloc[med].sort_values(by=['mark'])

,tutorname,mark,discipline,cluster
238,Финаев Вячеслав Вячеславович,3.000000,английский язык,238
125,Якубенко Наталья Валерьевна,7.000000,математика,125
316,Яшкина Анастасия Романовна,8.000000,биология,316
0,Рязанский Валерий Павлович,9.000000,математика,0
33,Литвинова Татьяна Валерьевна,9.000000,начальная школа,33
40,Шабалина Екатерина Сергеевна,9.000000,русский язык,40
474,Костин Анатолий Евгеньевич,9.250000,физика,474
165,Грипа Яна Николаевна,9.307692,логопеды,165
30,Зуева Татьяна Сергеевна,9.333333,английский язык,30
232,Хайрутдинов Руслан Илдусович,9.500000,химия,232


Можно сохранить результат

In [109]:
df_result.to_csv('./data/tutors_result.csv')